In [1]:
!npm install -g localtunnel --no-fund
!npm audit fix --force
!pip install flask-cors requests

import zipfile
import os
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import json
import threading
import shutil
import re
import time
import requests

app = Flask(__name__)
CORS(app)  # Permitir solicitudes CORS
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/', methods=['GET', 'OPTIONS'])
def home():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    return "Server is running", 200

@app.route('/upload', methods=['POST', 'OPTIONS'])
def upload_file():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    if 'file' not in request.files or 'project_name' not in request.form or 'ud_name' not in request.form:
        return 'No file part, project name, or unit name', 400
    file = request.files['file']
    project_name = request.form['project_name']
    ud_name = request.form['ud_name']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        project_folder = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
        ud_folder = os.path.join(project_folder, ud_name)
        os.makedirs(ud_folder, exist_ok=True)
        file_path = os.path.join(ud_folder, file.filename)
        file.save(file_path)
        process_uploaded_file(file_path, ud_folder)  # Llamar a la función para procesar el archivo
        return 'File uploaded and processed successfully', 200

@app.route('/rename_project', methods=['POST', 'OPTIONS'])
def rename_project():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    data = request.json
    old_name = data.get('old_name')
    new_name = data.get('new_name')
    if not old_name or not new_name:
        return 'Missing old or new project name', 400
    old_path = os.path.join(app.config['UPLOAD_FOLDER'], old_name)
    new_path = os.path.join(app.config['UPLOAD_FOLDER'], new_name)
    if os.path.exists(old_path):
        shutil.move(old_path, new_path)
        return 'Project renamed successfully', 200
    else:
        return 'Old project name not found', 404

@app.route('/rename_ud', methods=['POST', 'OPTIONS'])
def rename_ud():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    data = request.json
    project_name = data.get('project_name')
    old_name = data.get('old_name')
    new_name = data.get('new_name')
    if not project_name or not old_name or not new_name:
        return 'Missing project name, old name or new name', 400
    project_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
    old_path = os.path.join(project_path, old_name)
    new_path = os.path.join(project_path, new_name)
    if os.path.exists(old_path):
        shutil.move(old_path, new_path)
        return 'Unit renamed successfully', 200
    else:
        return 'Old unit name not found', 404

@app.route('/delete_ud', methods=['POST', 'OPTIONS'])
def delete_ud():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    data = request.json
    project_name = data.get('project_name')
    ud_name = data.get('ud_name')
    if not project_name or not ud_name:
        return 'Missing project name or unit name', 400
    project_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
    ud_path = os.path.join(project_path, ud_name)
    if os.path.exists(ud_path):
        shutil.rmtree(ud_path)
        return 'Unit deleted successfully', 200
    else:
        return 'Unit name not found', 404

@app.route('/delete_project', methods=['POST', 'OPTIONS'])
def delete_project():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    data = request.get_json()
    project_name = data['project_name']
    project_folder = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
    if os.path.exists(project_folder):
        import shutil
        shutil.rmtree(project_folder)
        return 'Project deleted successfully', 200
    else:
        return 'Project not found', 404

@app.route('/check_auth', methods=['GET', 'OPTIONS'])
def check_auth():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    return jsonify({"auth_status": "success"}), 200

def build_cors_preflight_response():
    response = jsonify({"message": "CORS preflight"})
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add("Access-Control-Allow-Headers", "*")
    response.headers.add("Access-Control-Allow-Methods", "GET, POST, OPTIONS, PUT, DELETE")
    return response

def process_uploaded_file(zip_path, extract_dir):
    os.makedirs(extract_dir, exist_ok=True)

    # Extraer archivo zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # Listar archivos extraídos
    extracted_files = os.listdir(extract_dir)
    print(extracted_files)

@app.route('/uploads/<path:path>')
def serve_file(path):
    return send_from_directory(UPLOAD_FOLDER, path)

@app.route('/modify', methods=['POST', 'OPTIONS'])
def modify_index():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    project_name = request.json.get('project_name')
    ud_name = request.json.get('ud_name')
    new_value = request.json.get('ntxCafCompressed')

    if not project_name or not ud_name:
        return jsonify({"status": "project_name or ud_name is missing"}), 400

    index_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name, ud_name, 'index_prof.html')

    # Añadir más información de depuración
    print(f"Project Name: {project_name}")
    print(f"Unit Name: {ud_name}")
    print(f"Index Path: {index_path}")

    if os.path.exists(index_path):
        with open(index_path, 'r') as index_file:
            content = index_file.read()

        # Usar una expresión regular para reemplazar la variable ntxCafCompressed
        updated_content = re.sub(r'var ntxCafCompressed = ".*";', f'var ntxCafCompressed = "{new_value}";', content)

        with open(index_path, 'w') as index_file:
            index_file.write(updated_content)

        return jsonify({"status": "success"})
    else:
        return jsonify({"status": "index file not found"}), 404

@app.route('/view_index', methods=['GET', 'OPTIONS'])
def view_index():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    project_name = request.args.get('project_name')
    ud_name = request.args.get('ud_name')

    if not project_name or not ud_name:
        return 'Missing project_name or ud_name', 400

    index_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name, ud_name, 'index_prof.html')

    if os.path.exists(index_path):
        with open(index_path, 'r') as index_file:
            content = index_file.read()
        return content, 200
    else:
        return 'index_prof.html not found', 404

@app.route('/list_projects', methods=['GET', 'OPTIONS'])
def list_projects():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()

    projects = []
    if os.path.exists(UPLOAD_FOLDER):
        projects = os.listdir(UPLOAD_FOLDER)

    return jsonify({"projects": projects}), 200

@app.route('/get_ntxCafCompressed', methods=['GET', 'OPTIONS'])
def get_ntxCafCompressed():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()

    project_name = request.args.get('project_name')
    ud_name = request.args.get('ud_name')

    if not project_name or not ud_name:
        return jsonify({"error": "Missing project_name or ud_name"}), 400

    index_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name, ud_name, 'index_prof.html')

    if not os.path.exists(index_path):
        return jsonify({"error": "index_prof.html not found"}), 404

    try:
        with open(index_path, 'r', encoding='utf-8') as index_file:
            content = index_file.read()

            # Usar una expresión regular para extraer el valor de ntxCafCompressed
            match = re.search(r'var ntxCafCompressed = "(.*?)";', content)
            if match:
                ntxCafCompressed_value = match.group(1)
                return jsonify({"ntxCafCompressed": ntxCafCompressed_value}), 200
            else:
                return jsonify({"error": "ntxCafCompressed variable not found"}), 404
    except Exception as e:
        return jsonify({"error": str(e)}), 500

def run_app():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_app)
thread.start()

time.sleep(5)  # Espera a que localtunnel se inicie

# Obtener la contraseña del túnel usando requests
response = requests.get('https://loca.lt/mytunnelpassword')
tunnel_password = response.text.strip()
print(f"Tunnel password: {tunnel_password}")

!lt --port 5000



added 22 packages, and audited 23 packages in 4s

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
npm WARN using --force Recommended protections disabled.
npm ERR! code ENOLOCK
npm ERR! audit This command requires an existing lockfile.
npm ERR! audit Try creating one first with: npm i --package-lock-only
npm ERR! audit Original error: loadVirtual requires existing shrinkwrap file

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2024-08-16T13_25_16_320Z-debug-0.log
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Tunnel password: 34.48.118.123
your url is: https://clean-radios-hunt.loca.lt


INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:26:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:26:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:26:55] "GET /check_auth HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:27:07] "GET /check_auth HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:27:08] "OPTIONS /list_projects HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:27:08] "GET /list_projects HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:27:36] "OPTIONS /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:28:21] "POST /upload HTTP/1.1" 200 -


['573_215_0_532', 'project.json', 'cmi5.json', 'components.json', 'resources', 'ud_files.zip', 'index_prof.html', 'trackingConfig.js']


INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:29:43] "GET /check_auth HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:29:43] "OPTIONS /list_projects HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2024 13:29:43] "GET /list_projects HTTP/1.1" 200 -


^C
